# Eigenvectors and Eigenvalues 


### Jacobi transformations

In [1]:
import torch
from torch.utils.cpp_extension import load

In [72]:
jacobi = load(name='jacobi',
             build_directory='./build',
             sources=['jacobi.cc'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++17'],
             verbose=False)

In [73]:
A_ = torch.randn((5,5)).double()
A = A_ + A_.t()
A

tensor([[ 0.7931, -0.8090, -1.2304, -3.1961, -0.7078],
        [-0.8090,  1.3235,  0.5319,  3.1358, -0.0623],
        [-1.2304,  0.5319, -1.9692,  0.1079, -2.0318],
        [-3.1961,  3.1358,  0.1079, -3.3581,  3.9510],
        [-0.7078, -0.0623, -2.0318,  3.9510,  1.0119]], dtype=torch.float64)

In [74]:
D, V = jacobi.symeig(A)

In [75]:
D

tensor([ 0.3504,  5.8521, -3.4011, -7.2766,  2.2763], dtype=torch.float64)

In [76]:
V

tensor([[ 0.6910, -0.4828,  0.3702,  0.2546,  0.2958],
        [ 0.6821,  0.4528, -0.0979, -0.2775, -0.4931],
        [-0.2370, -0.0199,  0.8471, -0.0823, -0.4680],
        [ 0.0074,  0.5430,  0.1063,  0.8327,  0.0192],
        [ 0.0311,  0.5164,  0.3528, -0.3975,  0.6708]], dtype=torch.float64)

In [77]:
Ds, Di = D.sort()
Ds

tensor([-7.2766, -3.4011,  0.3504,  2.2763,  5.8521], dtype=torch.float64)

In [79]:
V.t()[Di].t()

tensor([[ 0.2546,  0.3702,  0.6910,  0.2958, -0.4828],
        [-0.2775, -0.0979,  0.6821, -0.4931,  0.4528],
        [-0.0823,  0.8471, -0.2370, -0.4680, -0.0199],
        [ 0.8327,  0.1063,  0.0074,  0.0192,  0.5430],
        [-0.3975,  0.3528,  0.0311,  0.6708,  0.5164]], dtype=torch.float64)

In [80]:
e, vs = torch.linalg.eigh(A)

In [81]:
e

tensor([-7.2766, -3.4011,  0.3504,  2.2763,  5.8521], dtype=torch.float64)

In [82]:
vs

tensor([[-0.2546,  0.3702,  0.6910, -0.2958, -0.4828],
        [ 0.2775, -0.0979,  0.6821,  0.4931,  0.4528],
        [ 0.0823,  0.8471, -0.2370,  0.4680, -0.0199],
        [-0.8327,  0.1063,  0.0074, -0.0192,  0.5430],
        [ 0.3975,  0.3528,  0.0311, -0.6708,  0.5164]], dtype=torch.float64)

### Exercises 
**(1)** Let $A$ and $B$ be a real symmetric matrices, with $B$ positive definite. Implement an efficient routine to solve the generalised eigenvalue problem:
$$
A \cdot x = \lambda B \cdot x
$$

**(2)** Implement a routine to that attempts to solve the non-linear eigenvalue problem:
$$
(\lambda^2 A + \lambda B + C) \cdot x = 0
$$
over the reals, for any compatible set of square matrices.
